In [1]:
%load_ext autoreload
%autoreload 2

import rydanalysis
import rydanalysis as ra

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from scipy.signal import find_peaks
from scipy.stats import sem
from tqdm.notebook import tqdm
import arc
from pathlib import Path


%matplotlib widget

# Load data

In [2]:
path = Path.cwd().parent
path

WindowsPath('d:/Dropbox/Rydberg/Software/analysis/test_fitting')

In [3]:
data = ra.load_data(path / "raw_data", to_multiindex=True)
images = data.ryd_data.images
images.load()

<xarray.Dataset>
Dimensions:   (shot: 145, x: 256, y: 50)
Coordinates:
  * x         (x) float64 -547.9 -543.6 -539.3 -535.0 ... 539.3 543.6 547.9
  * y         (y) float64 -104.9 -100.6 -96.3 -92.02 ... 92.02 96.3 100.6 104.9
  * shot      (shot) MultiIndex
  - tmstp     (shot) datetime64[ns] 2021-05-20T16:05:09 ... 2021-05-20T16:11:19
  - tCAM      (shot) float64 0.01 0.0 0.0005 0.001 ... 0.007 0.0075 0.008 0.0085
Data variables:
    image_00  (shot, x, y) float32 428.0 647.0 686.0 792.0 ... 357.0 356.0 270.0
    image_01  (shot, x, y) float32 660.0 422.0 363.0 324.0 ... 221.0 216.0 225.0
    image_02  (shot, x, y) float32 264.0 269.0 267.0 275.0 ... 229.0 227.0 224.0
    image_03  (shot, x, y) float32 261.0 242.0 227.0 235.0 ... 223.0 220.0 216.0
    image_04  (shot, x, y) float32 238.0 267.0 283.0 279.0 ... 238.0 244.0 221.0
    image_05  (shot, x, y) float32 268.0 214.0 223.0 223.0 ... 220.0 216.0 216.0

# Analysis
## Analyse ground state images
### Choose square mask to remove edges of the images

In [4]:
ground_state_images = images.sel(tCAM=0)

background = ground_state_images.image_05.mean("tmstp")
absorption = ground_state_images.image_02
light = ground_state_images.image_04

## Select square mask
### Mask in imaging system 
polygon_mask_indices = [(-500, -80), (500, -80), (500, 90), (-500, 90)]
square_mask = light.isel(tmstp=0).polygon_mask.get_mask(polygon_mask_indices)

In [5]:
mean_light_image = light.mean("tmstp")

fig, axes = plt.subplots(2, figsize=(6, 4), sharex=True)

ax = axes[0]
mean_light_image.T.plot(ax=ax, robust=True)
ax.set_ylabel("y [$\mu$m]")

ax = axes[1]
mean_light_image.where(square_mask).T.plot(ax=ax, robust=True)
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Simple density calculation without pca

In [6]:
show_mask = True

if show_mask:
    mask = square_mask
else:
    mask = 1

fig, axes = plt.subplots(3, sharex=True)

absorption.mean("tmstp").where(mask).T.plot(ax=axes[0])
light.mean("tmstp").where(mask).T.plot(ax=axes[1])
background.where(mask).T.plot(ax=axes[2], robust=True)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
fig, ax = plt.subplots()

absorption.where(square_mask).mean(['x', 'y']).plot(ax=ax, label='absorption light')
light.where(square_mask).mean(['x', 'y']).plot(ax=ax, label='reference light')
ground_state_images.image_05.where(square_mask).mean(['x', 'y']).plot(ax=ax, label='background')
ax.legend()
ax.set_xlabel('timestamp')
ax.set_ylabel('signal')
fig.tight_layout()
fig.savefig('temporal_evolution_of_light_intenstity.png',dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# build imaging with averaged light images
atom_images = absorption.where(square_mask) - background
reference_images = light.mean("tmstp") - background

imaging_averaged = ra.AbsorptionImaging(atom_images, reference_images, t_exp=150e-3, binning=2)
density_averaged = imaging_averaged.density.isel(tmstp=0)

# build imaging with single light images
atom_images = absorption.where(square_mask) - background
reference_images = light.where(square_mask) - background

imaging_single = ra.AbsorptionImaging(atom_images, reference_images, t_exp=None, binning=2)
density_single = imaging_single.density.isel(tmstp=0)

# plot
fig, axes = plt.subplots(3, figsize=(6, 6), sharex=True)
density_single.T.plot(ax=axes[0], robust=True)
axes[0].set_title('no averge')

density_averaged.T.plot(ax=axes[1], robust=True)
axes[1].set_title('averged over reference')

(density_single - density_averaged).T.plot(ax=axes[2], robust=True)
axes[2].set_title('relative difference')
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()
fig.savefig('comparison_background.png', bbox_inches='tight', dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 ## Advanced analysis involving Principle component analysis (PCA)

In [15]:
# Select atom mask

atom_mask = images.elliptical_mask.get_mask(*[25, 11, 200, 200])

fig, axes = plt.subplots(2, figsize=(6, 3), sharex=True)
ax = axes[0]
density_averaged.where(atom_mask).T.plot(ax=ax, robust=True)
ax.set_ylabel("y [$\mu$m]")

ax = axes[1]
density_averaged.where(np.logical_not(atom_mask)).T.plot(ax=ax, robust=True)
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
background = images.image_05.mean("shot")
absorption = images.image_02
light = images.image_04

In [33]:
atom_images = absorption.where(square_mask) - background
pca = light.pca(n_components=2)
no_atoms_mask = np.logical_not(atom_mask)
reference_images = pca.find_references(absorption, no_atoms_mask) - background

imaging_pca = ra.AbsorptionImaging(atom_images, reference_images, t_exp=150e-3, binning=2)
density_pca = imaging_pca.density.isel(shot=0)

In [34]:
## compare both analysis methods

atom_mask = images.elliptical_mask.get_mask(*[-220, 0, 350, 200])

fig, axes = plt.subplots(3, figsize=(6, 4), sharex=True)
density_pca.T.plot(ax=axes[0], robust=True);
axes[0].set_title('with pca')
axes[0].set_ylabel("y [$\mu$m]")

density_single.T.plot(ax=axes[1], robust=True);
axes[1].set_title('no pca')
axes[1].set_ylabel("y [$\mu$m]")


(density_pca - density_single).T.plot(ax=axes[2], robust=True);
axes[2].set_title('relative difference')
axes[2].set_ylabel("y [$\mu$m]")

fig.tight_layout()
fig.tight_layout()
fig.savefig('comparison_pca.png', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The analysis with pca is problematic for multiple reasons:

- The imaging timing was not correct and we still see some remanences of the shifting on the andor sensor
- The background picture is not useful
- The size of the atom cloud is so large, that the remaining space for pca is too little

The pca tries to avoid these features and does weird things. We will continue without pca

In [40]:
x, y = np.sort([1, 4])
y

4

In [41]:
image = ra.Image(density_pca)
%time fit_params = image.fitgaussian
fig, ax = plt.subplots(figsize=(8, 2))

image.plot_fit(ax=ax)
fit_params

Wall time: 34 ms


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

height       1.983214e+11
center_x     1.098154e+01
center_y     2.662139e+01
sig_x        5.901240e+01
sig_y        8.360091e+01
N            6.147573e+03
density3d    1.094058e+09
dtype: float64

In [57]:
fit_results = []
for tmstp, image in tqdm(imaging_pca.density.groupby("shot")):
    i = ra.Image(image)
    fit_results.append(i.fitgaussian)
fit_results = pd.concat(fit_results, axis=1).T
fit_results.index = imaging_pca.density.shot.to_index()
# fit_results.index.name = "tmstp"

  0%|          | 0/145 [00:00<?, ?it/s]

In [58]:
fig, axes = plt.subplots(3, figsize=(6, 5), sharex=True)

ax = axes[0]
fit_results[["center_x", "center_y"]].plot(ax=ax)
ax.set_ylabel("position [$\mu$m]")

ax = axes[1]
fit_results[["sig_x", "sig_y"]].plot(ax=ax)
ax.set_ylabel("position [$\mu$m]")

ax = axes[2]
fit_results["density3d"].plot(ax=ax)
ax.set_ylim(bottom=0)
ax.set_ylabel("density [1/cm³]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'density [1/cm³]')

In [61]:
fig, ax = plt.subplots()
fit_results.groupby("tCAM").mean().density3d.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='tCAM'>